In [1]:
import os
import pickle
from tqdm import tqdm
from math import log
from nltk import PorterStemmer

In [2]:
folder = 'documents/'

In [3]:
k = 2
b = 0.75

stemmer = PorterStemmer()

N = 0
sum_len = 0

dl = {}
tf = {}
bm = {}
doc_desc = {}

text_start = '<span class="descriptor">Abstract:</span>'
url_start = '<meta name="citation_pdf_url" content="'
url_end = '" />'
title_start = '<meta name="citation_title" content="'
title_end = '" />'

for d in tqdm(os.listdir(folder)):
    with open(folder + d) as f:
        dl[d] = 0
        in_text = False
        for line in f.readlines():
            if url_start in line:
                url = line[len(url_start):-len(url_end)-1].replace('pdf', 'abs')
            if title_start in line:
                title = line[len(title_start):-len(title_end)]
            if text_start in line:
                in_text = True
                line = line[len(text_start):]
                content = ''
            if not in_text:
                continue
            if '</blockquote>' in line:
                break
            content += line
            terms = [stemmer.stem(word).encode("utf8") for word in line.lower().split(' ')
                                                         if len(word) > 1 and word.isalpha()]
            dl[d] += len(terms)
            for t in terms:
                if t in bm:
                    if d not in bm[t]:
                        bm[t].append(d)
                else:
                    bm[t] = [d]
                if (t, d) in tf:
                    tf[(t, d)] += 1
                else:
                    tf[(t, d)] = 1
        doc_desc[d] = {'title': title, 'url': url, 'content': content}
        N += 1
        sum_len += dl[d]
    
dl_ave = sum_len / N

100%|██████████| 2911/2911 [00:10<00:00, 276.83it/s]


In [4]:
dl_ave

63.168670559945035

In [5]:
N

2911

In [6]:
len(bm.keys())

7439

In [7]:
list(tf.keys())[:10]

[(b'in', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'thi', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'we', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'develop', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'new', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'method', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'to', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'produc', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'explicit', '5e43797895975c03778e73ceaf0e9ebd'),
 (b'formula', '5e43797895975c03778e73ceaf0e9ebd')]

In [8]:
for t in tqdm(bm):
    scores = []
    df_t = len(bm[t])
    for d in bm[t]:
        scores.append(log(N / df_t) * ((k + 1) * tf[(t, d)]) / (k * (1 - b + b * dl[d] / dl_ave) + tf[(t, d)]))
    bm[t] = list(zip(scores, bm[t]))

100%|██████████| 7439/7439 [00:00<00:00, 30084.54it/s]


In [9]:
with open('bm.pickle', 'wb') as bm_file:
    pickle.dump(bm, bm_file)

In [10]:
max_len = 0
avg_len = 0

for t in tqdm(bm):
    max_len = max(max_len, len(bm[t]))
    avg_len += len(bm[t])
    
avg_len /= len(bm.keys())

100%|██████████| 7439/7439 [00:00<00:00, 628452.86it/s]


In [11]:
avg_len, max_len

(14.965586772415648, 1661)